# 1. Install and Import Dependencies

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [21]:
import numpy as np
import pandas as pd

# 2. Instantiate Model

In [2]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/Users/ayseljafarova/anaconda3/envs/newenv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/Users/ayseljafarova/anaconda3/envs/newenv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

# 3. Encode and Calculate Sentiment

In [3]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [4]:
result = model(tokens)

In [5]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [6]:
int(torch.argmax(result.logits))+1

4

# 4. Collect Reviews

In [19]:
r = requests.get('https://www.trustpilot.com/review/www.dugood.org')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*typography_body-l__KUYFJ.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [20]:
reviews

['4.9',
 '3,933 total',
 'The customer service was amazing. The loan officer Allison Medina was exceptional and would recommend her to anyone. I have had other credit unions but this one beats them all. We were at Rave but gladly switched to here. Great service and awesome credit union and the employees that work here.',
 'I applied for an auto refinancing loan and it was processed the same day. Tracy at the Beaumont Eastex Freeway location was walked me through the process and was extremely helpful, friendly and professional. I couldn’t have been more pleased. ',
 'I financed my car through DuGood and I needed to have a account through them. Decided switching banks was easier then having multiple accounts open. Talked with Christy and she was very nice and helpful. She walked me through the whole process and explained everything in a simple and easy way to understand. I got more benefits then I did with my other bank and staff was so much nicer. Thank you Christy! Would recommend doin

# 5. Load Reviews into DataFrame and Score

In [22]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [24]:
df['review'].iloc[2]

'The customer service was amazing. The loan officer Allison Medina was exceptional and would recommend her to anyone. I have had other credit unions but this one beats them all. We were at Rave but gladly switched to here. Great service and awesome credit union and the employees that work here.'

In [14]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [25]:
sentiment_score(df['review'].iloc[1])

1

In [26]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [30]:
df.tail()

,review,sentiment
20,There are so many amazing people on staff that...,5
21,Loan officer Tara was very helpful. She answer...,5
22,Panacea Financial,5
23,Network Capital Funding,4
24,AARDY,5


In [28]:
df.shape

(25, 2)

In [29]:
df['review'].iloc[3]

'I applied for an auto refinancing loan and it was processed the same day. Tracy at the Beaumont Eastex Freeway location was walked me through the process and was extremely helpful, friendly and professional. I couldn’t have been more pleased. '